In [1]:
import torch


---

# 📌 PyTorch 메모리 포맷 정리

## 1. 기본 개념

* **메모리 포맷(memory_format)** = 텐서가 **메모리에 어떤 순서로 저장되어 있는지**를 정의
* CNN 연산 속도 최적화에서 중요 (특히 GPU, cuDNN)

---

## 2. 주요 옵션

### 🔹 `torch.contiguous_format`

* PyTorch 기본 포맷 = **NCHW** (Batch, Channel, Height, Width)
* 메모리에 채널 단위로 모아서 저장
* `x.is_contiguous()` → True면 이 포맷

### 🔹 `torch.channels_last`

* **NHWC** (Batch, Height, Width, Channel)
* 픽셀 단위로 RGB가 연속 저장됨
* Conv2d에서 GPU 최적화 효과 ↑
* 사용법:

  ```python
  x = x.to(memory_format=torch.channels_last)
  model = model.to(memory_format=torch.channels_last)
  ```

### 🔹 `torch.channels_last_3d`

* **NDHWC** (Batch, Depth, Height, Width, Channel)
* 3D ConvNet (영상, CT/MRI 등) 최적화
* Conv3d 연산에 효과적

### 🔹 `torch.preserve_format`

* `clone`, `to`, `detach` 등으로 텐서를 복사할 때
* **입력 텐서의 메모리 포맷 그대로 유지**
* 예: `channels_last` 입력을 그대로 복제할 때 사용

---

## 3. 왜 중요한가?

* CNN 연산에서는 **픽셀 단위로 채널(RGB 등)을 동시에 불러와야 함**
* `contiguous_format(NCHW)` → 같은 픽셀의 채널 값이 메모리에 흩어져 있음 (비효율적)
* `channels_last(NHWC)` → 같은 픽셀의 채널이 연속 저장 → **GPU가 직선으로 빠르게 읽음(coalesced access)**
* 결과적으로 Conv 속도가 빨라지고, 특히 AMP(FP16)와 같이 쓰면 효과 극대화

---

## ✅ 한 줄 요약

* **contiguous_format** = 기본 (NCHW)
* **channels_last** = 2D CNN GPU 최적화 (NHWC)
* **channels_last_3d** = 3D CNN GPU 최적화 (NDHWC)
* **preserve_format** = 텐서 복제 시 원래 포맷 유지

---



In [3]:
# 입력을 channels_last로 만든 상태
x = torch.randn(8, 3, 224, 224, device="cpu").to(memory_format=torch.channels_last)

# 그냥 clone() → 기본 contiguous_format으로 바뀔 수 있음
y1 = x.clone()
print("default clone stride:", y1.stride())

# preserve_format 사용 → 입력과 같은 메모리 포맷 유지
y2 = x.clone(memory_format=torch.preserve_format)
print("preserve stride:", y2.stride())

print("same format?", x.stride() == y2.stride())  # True

default clone stride: (150528, 1, 672, 3)
preserve stride: (150528, 1, 672, 3)
same format? True
